In [ ]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
# leggo il dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# prima vista generale sui dati

In [ ]:
train.head()

In [ ]:
# correggo i nomi delle colonne che hanno errori di battitura
train.rename(columns = {'No-show':'NoShow','Hipertension':'Hypertension','Handcap':'Handicap'},inplace=True)
test.rename(columns = {'No-show':'NoShow','Hipertension':'Hypertension','Handcap':'Handicap'},inplace=True)
train.columns

In [ ]:
train.describe()

In [ ]:
# controllo il numero di valori mancanti di train
print(train.isnull().sum())

In [ ]:
# controllo il numero di valori mancanti di test
print(test.isnull().sum())

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
# conto quanti si sono presentati all'appuntamento e quanti no
noShowTr = train['NoShow'].value_counts()
print(noShowTr)

print("\n-------------\n")

noShowTe = test['NoShow'].value_counts()
print(noShowTe)

In [ ]:
# codifico NoShow usando 0 per 'No' e 1 per 'Si'
train['NoShow'].replace('Yes', 1, inplace=True)
train['NoShow'].replace('No', 0, inplace=True)

test['NoShow'].replace('Yes', 1, inplace=True)
test['NoShow'].replace('No', 0, inplace=True)

In [ ]:
# conto quanti maschi e femmine ci sono
genderTr = train['Gender'].value_counts()
print(genderTr)

print("\n-------------\n")

genderTe = test['Gender'].value_counts()
print(genderTe)

In [ ]:
# codifico Gender usando 1 per 'M' e 0 per 'F'

train['Gender'].replace('M', 1, inplace=True)
train['Gender'].replace('F', 0, inplace=True)

test['Gender'].replace('M', 1, inplace=True)
test['Gender'].replace('F', 0, inplace=True)

In [ ]:
# converto 'ScheduledDay' e 'AppointmentDay' in datetime
train.ScheduledDay = train.ScheduledDay.apply(np.datetime64)
train.AppointmentDay = train.AppointmentDay.apply(np.datetime64)

test.ScheduledDay = test.ScheduledDay.apply(np.datetime64)
test.AppointmentDay = test.AppointmentDay.apply(np.datetime64)

In [ ]:
# creo una variabile 'WaitingTime' che rappresenta il tempo di attesa
# dalla prenotazione della visita alla visita stessa
train['WaitingTime'] = (train.AppointmentDay.dt.date - train.ScheduledDay.dt.date).dt.days
test['WaitingTime'] = (test.AppointmentDay.dt.date - test.ScheduledDay.dt.date).dt.days

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
# guardo i valori unici
print("Valori unici in 'NoShow' => {}".format(train.NoShow.unique()))
print("Valori unici 'Gender' => {}".format(train.Gender.unique()))
print("Valori unici in 'Age' => {}".format(train.Age.unique()))
print("Valori unici in 'Neighbourhood' => {}".format(train.Neighbourhood.unique()))
print("Valori unici in 'Scholarship' => {}".format(train.Scholarship.unique()))
print("Valori unici in 'Hypertension' => {}".format(train.Hypertension.unique()))
print("Valori unici in 'Diabetes' => {}".format(train.Diabetes.unique()))
print("Valori unici in 'Alcoholism' => {}".format(train.Alcoholism.unique()))
print("Valori unici in 'Handicap' => {}".format(train.Handicap.unique()))
print("Valori unici in 'SMS_received' => {}".format(train.SMS_received.unique()))
print("Valori unici in 'ScheduledDay' => {}".format(train.ScheduledDay.unique()))
print("Valori unici in 'AppointmentDay' => {}".format(train.AppointmentDay.unique()))
print("Valori unici in 'WaitingTime' => {}".format(train.WaitingTime.unique()))

In [ ]:
# tratto le età superiori a 100 e inferiori a 0 come valori anomali, quindi li rimuovo
train = train[(train.Age >= 0) & (train.Age <= 100)]
test = test[(test.Age >= 0) & (test.Age <= 100)]

In [ ]:
# osservo in che modo ogni feature si relaziona a quella principale 'NoShow'
features = ['Gender','Scholarship','Hypertension','Diabetes','Alcoholism','Handicap','SMS_received']
for f in features:
    print(train.groupby(f)['NoShow'].mean())

In [ ]:
# provo a rappresentare attraverso la variabile 'NumMissedApp', utilizzando la somma 
# cumulativa, il numero di appuntamenti che il paziente ha saltato nel passato, dato che penso
# che possano essere determinanti ai fini di prevedere la presenza alle visite successive
train['NumMissedApp'] = train.groupby('PatientId')['NoShow'].apply(lambda x: x.cumsum())
test['NumMissedApp'] = test.groupby('PatientId')['NoShow'].apply(lambda x: x.cumsum())

In [ ]:
# rimuovo le features
train.drop(['PatientId','AppointmentID','ScheduledDay','AppointmentDay','Neighbourhood'], axis=1, inplace=True)
test.drop(['PatientId','AppointmentID','ScheduledDay','AppointmentDay','Neighbourhood'], axis=1, inplace=True)

In [ ]:
# test e training

In [ ]:
x_train = train.drop(['NoShow'], axis=1) 
y_train = train['NoShow']           
      
x_test = test.drop(['NoShow'], axis=1)   
y_test = test['NoShow']

In [ ]:
# utilizzo la logistic regression 
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
accuracy = accuracy_score(y_pred, y_test) * 100

print("{:.2f}".format(accuracy))